In [6]:
!pip install transformers
!pip install sentencepiece
!pip install -U sentence-transformers

In [1]:
import torch
import pandas as pd

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## New Code

In [2]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
testdf=pd.read_csv("Preprocessed_Train.csv",header=0)
corpus = testdf.Text.tolist()
# Approx 3min +
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 112


In [3]:
from tqdm import tqdm

results_df = pd.DataFrame()
idx = []
score = []
top_k = []

for cur_doc,query_embed in tqdm(enumerate(corpus_embeddings), total=len(corpus)):
  hits = util.semantic_search(query_embed, corpus_embeddings, top_k=10+1)
  hits = hits[0]
  idx += [cur_doc] * 10
  score += [x['score'] for x in hits[1:]]
  top_k += [x['corpus_id'] for x in hits[1:]]

results_df['idx'] = idx
results_df['score'] = score
results_df['top_k'] = top_k

results_df.head(30)

ERROR! Session/line number was not unique in database. History logging moved to new session 113
100%|██████████| 63616/63616 [03:56<00:00, 268.65it/s]


,idx,score,top_k
0,0,0.787706,6261
1,0,0.772041,57615
2,0,0.734515,26360
3,0,0.733349,62450
4,0,0.732962,33945
5,0,0.718994,48580
6,0,0.714439,44878
7,0,0.707467,16893
8,0,0.705440,8520
9,0,0.699479,55796


In [8]:
def print_candidate_results(candidate):
    print(corpus[candidate][:200] + '....')
    print('-' * 30)
    candidate_results = results_df.loc[results_df['idx']==candidate][['score','top_k']]
    candidate_results['text'] = [corpus[x] for x in candidate_results['top_k']]
    print(candidate_results)

In [9]:
print_candidate_results(0)

Report War Looms Hundreds American Troops CIA Operatives Have Entered SyriaMac Slavo If following Frances second largest newspaper Le Figaro accurate must assume war foregone conclusion. According inf....
------------------------------
      score  top_k                                               text
0  0.787706   6261  CIA US State Dept. Sending Weapons Supplies Sy...
1  0.772041  57615  Skeptics American efforts arm anti government ...
2  0.734515  26360  Americas Covert Re Invasion IraqTony Cartalucc...
3  0.733349  62450  Fighting Erupts Lebanon US Israeli Saudi funde...
4  0.732962  33945  Surreal US Troops Stage Jordan Defend Al Qaeda...
5  0.718994  48580  Confirmed US Claims Against Syria There Eviden...
6  0.714439  44878  CIA Press Conference Looks Like SNL Skit Offic...
7  0.707467  16893  MANAMA Bahrain The United States sending 200 a...
8  0.705440   8520  U.S. Will Take Action In Syria Regardless Supp...
9  0.699479  55796  ISIS In Iraq The Path To IranBrandon Turbevi

In [10]:
print_candidate_results(2)

Neo Con Hypocrites Leverage Human Rights Against ChinaTony Cartalucci Contributor It illustrates unified agenda Neo Cons like Max Boot liberals like Hillary Clinton Obama administration. It also repre....
------------------------------
       score  top_k                                               text
20  0.543971  11689  McCain Lvy Openly Conspire Against World Peace...
21  0.539870  52660  In Syria Obama Fights Bushs WarsSyrian Opposit...
22  0.538783  62001  Road White House New Boss Will Be Same Old Bos...
23  0.538020  59327  G7 Protests Much To Do About NothingBy 21st Ce...
24  0.536241  56504  The Extravagant Hypocrisy Obamas Foreign Polic...
25  0.528523  29214  Syrias War Was Only Ever The BeginningBy Tony ...
26  0.524119  19472  From Syria To Asia To Russia Terror Network Or...
27  0.523715  41176  US Now Admits Funding Occupy CentralTony Carta...
28  0.519299  54813  In days immediately following presidential ele...
29  0.516967  25224  As momentum builds toward 2016 el

In [11]:
print_candidate_results(10)

A couple years ago heard news 50 vertebrate species disappeared 40 years. On Thursday greeted news 2020 figure likely rise 66 vertebrates. It wonder conservationists shouting. It wonder desperate get ....
------------------------------
        score  top_k                                               text
100  0.616194  20909  A bird small fragile thing. It chirrups flutte...
101  0.611784  25322  This week biggest stories If I said beautiful ...
102  0.608578   3899  Nearly 40 Million Bee Holocaust CanadaHeather ...
103  0.605432  27356  Brexit may caused anti immigration buzz travel...
104  0.599455  32038  At first glance bafflingly bad summer butterfl...
105  0.597969  18158  Whats Causing All Wildlife Die offs Around Wor...
106  0.591994  60833  The last red admirals commas feeding ivy flowe...
107  0.589933  56444  Hoping release updated image might generate le...
108  0.582304  28950  UN Grocery List Bread Milk Grubs Beetles Maggo...
109  0.580667  19783  UK Home Garden Diversi

In [12]:
from tqdm import trange

true_labels = list(testdf['Label'])
pred_labels = []
for i in trange(len(corpus)):
    list_i=results_df.loc[results_df['idx']==i, 'top_k'].tolist()
    label_i=int(round(sum([testdf.Label[x] for x in list_i])/len(list_i),0))
    pred_labels.append(label_i)

100%|██████████| 63616/63616 [01:25<00:00, 741.14it/s]


In [13]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, pred_labels, target_names=['Fake', 'Real']))

              precision    recall  f1-score   support

        Fake       0.87      0.83      0.85     31025
        Real       0.84      0.88      0.86     32591

    accuracy                           0.86     63616
   macro avg       0.86      0.86      0.86     63616
weighted avg       0.86      0.86      0.86     63616



## Old Code

In [6]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Single list of sentences - Possible tens of thousands of sentences
testdf=pd.read_csv("drive/MyDrive/Preprocessed_Train.csv",header=0)
sentences=testdf.Text.tolist()

paraphrases = util.paraphrase_mining(model, sentences, show_progress_bar=True, top_k=10)
#for paraphrase in paraphrases[0:1]:
    #score, i, j = paraphrase
    #print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], j, score))
    #print(paraphrase)
#print(paraphrases)

In [7]:
for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(i, j, score))
    #print(paraphrase)
#print(paraphrases)

97 		 9397 		 Score: 1.0000
97 		 29696 		 Score: 1.0000
97 		 43872 		 Score: 1.0000
551 		 52983 		 Score: 1.0000
7481 		 22796 		 Score: 1.0000
7481 		 58736 		 Score: 1.0000
9397 		 29696 		 Score: 1.0000
9397 		 43872 		 Score: 1.0000
22796 		 58736 		 Score: 1.0000
29696 		 43872 		 Score: 1.0000


In [13]:
resultsdf=pd.DataFrame(paraphrases, columns=["Score","article","top K"])
resultsdf.sort_values(by=["article", "Score"], ascending=[True,False], inplace=True, kind='quicksort', key=None, ignore_index=True)

In [41]:
resultsdf.head(30)

,Score,article,top K
0,0.787705,0,6261
1,0.772042,0,57615
2,0.734515,0,26360
3,0.733349,0,62450
4,0.732962,0,33945
5,0.718994,0,48580
6,0.714439,0,44878
7,0.707467,0,16893
8,0.705440,0,8520
9,0.699480,0,55796


In [39]:
candidate = 2
print(corpus[candidate])

Neo Con Hypocrites Leverage Human Rights Against ChinaTony Cartalucci Contributor It illustrates unified agenda Neo Cons like Max Boot liberals like Hillary Clinton Obama administration. It also represents one bizarre facet unfolding charade US attempting use China ahead talks aimed exacting concessions everything economic policy yielding West Syria. Max Boot makes astounding comments regarding recent row United States long cultivated activist Chen Guangcheng also known blind activist government China United States preparing meet talks. Clearly Max Boot represents cadre individuals reality greatest violators human rights world thus column Commentary titled Standing Up Human Rights China astounding transparent example exploitation otherwise noble principles. It mentioned NEDs board directors like Boot card carrying Neo Conservatives pro war represent largest corporate financier interests Earth whose policies actions directly contradict NEDs alleged mission supporting freedom around worl

In [40]:
for x in resultsdf.loc[resultsdf['article']==candidate, 'top K']:
  print(x)

0.5684084296226501 3 Friends local man Charles Wyatt intolerant asshole unrepentantly despises non Caucasians confirmed Tuesday deeply bigoted man makes best barbecue around. Oh man Charlies absolute master grill said Wyatts next door neighbor Mark Hawthorne like racist pricks exclusively white friends long enjoyed hateful small minded mans expertly prepared grilled chicken spare ribs beef brisket. First marinates everything homemade North Carolinastyle sauce slow cooks hours meat falls bone. I swear nobody quite like Charlie. Sources additionally confirmed Wyatts tantalizing barbecue dishes perfectly complemented cooking wife Sandy vicious anti Semite makes best macaroni salad youve ever life.
0.6133176684379578 4 There downtown vibe newest 21c Museum Hotel. Contemporary artworks fill public gallery spaces former factory building. The restaurant features open kitchen white subway tiles stout concrete pillars. Guest rooms look like lofts platform beds floor ceiling drapes draw across e

In [ ]:
#print(resultsdf.iloc[1000:2000])
print(len(resultsdf))
print(len(sentences))

374963
63616


In [ ]:
countdf=resultsdf.groupby('article').count()
countdf[countdf==10].count()

Score    10195
top K    10195
dtype: int64

In [ ]:
empty=[]
for i in range(len(sentences)):
    list_i=resultsdf.loc[resultsdf['article']==i, 'top K'].tolist()
    if list_i==[]:
        empty.append(i)

In [ ]:
results_label=[]
no=0
for i in range(len(sentences)):
    if i in empty:
        results_label.append(testdf.Label[i])
    else:
        list_i=resultsdf.loc[resultsdf['article']==i, 'top K'].tolist()
        label_i=int(round(sum([testdf.Label[x] for x in list_i])/len(list_i),0))
        results_label.append(label_i)
        no+=1
testdf['results']=results_label
print(testdf)

                                                    Text  Label  results
0      Report War Looms Hundreds American Troops CIA ...      0        0
1      I walked preview artist Pedro Reyes Creative T...      1        1
2      Neo Con Hypocrites Leverage Human Rights Again...      0        0
3      A giant beef roast holidays many de rigueur. S...      1        0
4      Seeking appeal guests desire new experiences s...      1        1
...                                                  ...    ...      ...
63611  BOOM Top Ebola Scientist Joked About Culling H...      0        0
63612  According study released Wednesday Center Medi...      0        1
63613  Following speech United Nations General Assemb...      0        0
63614  Interuniversity Microelectronics Centre Imec w...      1        1
63615  Indian Point Nuclear Power Plant A Ticking Tim...      0        0

[63616 rows x 3 columns]


In [ ]:
wrong=0
for i in range(len(sentences)):
    if i in empty:
        next
    if testdf.iloc[i]['Label']!=testdf.iloc[i]['results']:
        wrong+=1
print(wrong/no)

0.1543424058026607
